In [1]:
import pandas as pd
from epilepsy_prediction import load_prediction_model, data_preprocess,fuse_string,evaluate_model,load_imputation_model
from epilepsy_prediction.imputation import column_imputer
from copy import deepcopy

# How to predict

Loading the model, we can select between: Carbamazepine, Gabapentin, Lamotrigine, Levetiracetam, Oxcarbazepine, Phenytoin,Topiramate,Valproate. 

In [2]:
def load_data(change_name_file,data_file_name,outcome_col="outcome"):
    data_df = pd.read_csv(data_file_name, index_col=0)
    new_name_df = pd.read_csv(change_name_file,index_col=0)
    data_df.rename(new_name_df['marketscan'].to_dict(),inplace=True,axis=1)
    y_train = data_df.loc[data_df["test_train"]=="train",outcome_col]
    X_train = data_df.loc[data_df["test_train"]=="train",~( data_df.columns.isin([outcome_col,"test_train"]))]
    y_test = data_df.loc[~(data_df["test_train"]=="train"),outcome_col]
    X_test = data_df.loc[~(data_df["test_train"]=="train"),~ (data_df.columns.isin([outcome_col,"test_train"]))]
    return X_train,y_train,X_test,y_test


In [5]:
change_name_file=r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\dictionary_19.01.23.csv"
data_file_name = r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\Data\test_train_epilepsy_19.01.23.csv"
X_train,y_train,X_test,y_test = load_data(change_name_file,data_file_name, outcome_col="outcome_any")

In [6]:
drug_name="carbamazepine"
clf = load_prediction_model(drug_name)
X = data_preprocess(X_train,impute_drug=drug_name,replace_mean=True)
Xs = data_preprocess(X_test,impute_drug=drug_name,replace_mean=True)

c:\users\anastassia kolde\documents\github\unstableepilepsy\epilepsy_prediction\epilepsy_prediction\utils.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[col] = imputed_values[col]
c:\users\anastassia kolde\documents\github\unstableepilepsy\epilepsy_prediction\epilepsy_prediction\utils.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[col] = imputed_values[col]
c:\users\anastassia kolde\documents\github\unstableepilepsy\epilepsy_prediction\epilepsy_prediction\utils.py:43: PerformanceWarning: DataFrame is highly fragm

And we can also use the model for training using it's fit method. 

In [7]:
new_clf = deepcopy(clf)
new_clf.fit(X,y_train)
#new_clf.save_model('../models/prediction/est_carba_p.xgb')

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', gamma=0.4, gpu_id=-1,
              grow_policy='depthwise', importance_type='gain',
              interaction_constraints='', learning_rate=0.05, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=4, max_leaves=0,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, ...)

In [8]:
eval = evaluate_model(clf,new_clf,Xs,y_test)
rep = fuse_string(eval)

In [10]:
print(rep)

auc1: 0.57 (upper: 0.65, lower: 0.48) 
 auc2: 0.65 (upper: 0.73, lower: 0.56) 
 mcnemar_test: p-value 1.000 
 delong_test: p-value 0.127 
 


# Outcome primary

In [11]:
data_file_name = r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\Data\test_train_epilepsy_19.01.23.csv"
X_train,y_train,X_test,y_test = load_data(change_name_file,data_file_name, outcome_col="outcome_primary")
drug_name="carbamazepine"
clf = load_prediction_model(drug_name)
X = data_preprocess(X_train,impute_drug=drug_name)
Xs = data_preprocess(X_test,impute_drug=drug_name)
new_clf = deepcopy(clf)
new_clf.fit(X,y_train)
#new_clf.save_model('../models/prediction/est_carba_new_out.xgb')
eval = evaluate_model(clf,new_clf,Xs,y_test)
rep = fuse_string(eval)

c:\users\anastassia kolde\documents\github\unstableepilepsy\epilepsy_prediction\epilepsy_prediction\utils.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[col] = imputed_values[col]
c:\users\anastassia kolde\documents\github\unstableepilepsy\epilepsy_prediction\epilepsy_prediction\utils.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[col] = imputed_values[col]
c:\users\anastassia kolde\documents\github\unstableepilepsy\epilepsy_prediction\epilepsy_prediction\utils.py:43: PerformanceWarning: DataFrame is highly fragm

In [12]:
print(rep)

auc1: 0.54 (upper: 0.68, lower: 0.39) 
 auc2: 0.55 (upper: 0.68, lower: 0.41) 
 mcnemar_test: p-value 1.000 
 delong_test: p-value 0.890 
 


# Using the regressor imputation

In [15]:
#col_imputer = load_imputation_model("col_carbamazepine")
regress_imputer = load_imputation_model("regress_imputer_carbamazepine.pkl")

C:\ProgramData\AnacondaEnvs\epilepsy\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator LassoCV from version 1.1.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\ProgramData\AnacondaEnvs\epilepsy\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.1.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\ProgramData\AnacondaEnvs\epilepsy\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GridSearchCV from version 1.1.2 when using version 1.2.0. This might lead to breaking code or inval

In [14]:
X = data_preprocess(X_train,impute_drug=drug_name,replace_mean=False)
Xs = data_preprocess(X_test,impute_drug=drug_name,replace_mean=False)
print('Do we have nan value !:',any(X.isnull()))
X = regress_imputer.transform(X)
Xs = regress_imputer.transform(Xs)
new_clf = deepcopy(clf)
new_clf.fit(X,y_train)
#new_clf.save_model('../models/prediction/est_carba_new_impute.xgb')
eval = evaluate_model(clf,new_clf,Xs,y_test)
rep = fuse_string(eval)

Do we have nan value !: True


ValueError: Input X contains NaN.
LassoCV does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
print(rep)